In [1]:
import org.apache.spark.sql.SparkSession
spark.stop()


Intitializing Scala interpreter ...

Spark Web UI available at http://EM2021002716.bosonit.local:4040
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1619704635879)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession


In [2]:
val spark = SparkSession
    .builder()
    .appName("NASA web logs")
    .master("local")
    .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@58840c9b


 **Comenzamos leyendo los datasets de los weblogs que tenemos como .txt**

In [3]:
val file = "/Users/mario.serrano/Desktop/NASA/datasets/*.txt"
val logsDF = spark.read.text(file)

file: String = /Users/mario.serrano/Desktop/NASA/datasets/*.txt
logsDF: org.apache.spark.sql.DataFrame = [value: string]


In [4]:
logsDF.show(5, false)

+--------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------+
|in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839|
|uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0                                                   |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0                          |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0                        |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0                           |


**Teniendo el dataframe con una sola columna, ahora toca recoger los valores que queremos para nuestra tabla, lo haré mediante expresiones regulares**

In [5]:
val regex = 
"""(\S+)\s(-|\S+)\s(-|\S+)\s\[(\S+)\s\-\d{4}\]\s\"(GET|POST|PUT|TRACE|HEAD)?\s*(\S+|\S+\s*\S*)\s*(HTTP\S+\s*\S*)?\"\s(\d{3})\s(-|\d+)"""

val parsedDF = logsDF
    .select(regexp_extract($"value",regex,1).as("host"),
            regexp_extract($"value",regex,4).as("date"),
            regexp_extract($"value",regex,5).as("method"),
            regexp_extract($"value",regex,6).as("resource"),
            regexp_extract($"value",regex,7).as("protocol"),
            regexp_extract($"value",regex,8).cast("int").as("status"),
            regexp_extract($"value",regex,9).cast("int").as("bytes")
           )
parsedDF.show(10, false)
parsedDF.printSchema

+---------------------------+--------------------+------+-----------------------------------------------+--------+------+-----+
|host                       |date                |method|resource                                       |protocol|status|bytes|
+---------------------------+--------------------+------+-----------------------------------------------+--------+------+-----+
|in24.inetnebr.com          |01/Aug/1995:00:00:01|GET   |/shuttle/missions/sts-68/news/sts-68-mcc-05.txt|HTTP/1.0|200   |1839 |
|uplherc.upl.com            |01/Aug/1995:00:00:07|GET   |/                                              |HTTP/1.0|304   |0    |
|uplherc.upl.com            |01/Aug/1995:00:00:08|GET   |/images/ksclogo-medium.gif                     |HTTP/1.0|304   |0    |
|uplherc.upl.com            |01/Aug/1995:00:00:08|GET   |/images/MOSAIC-logosmall.gif                   |HTTP/1.0|304   |0    |
|uplherc.upl.com            |01/Aug/1995:00:00:08|GET   |/images/USA-logosmall.gif                      

regex: String = (\S+)\s(-|\S+)\s(-|\S+)\s\[(\S+)\s\-\d{4}\]\s\"(GET|POST|PUT|TRACE|HEAD)?\s*(\S+|\S+\s*\S*)\s*(HTTP\S+\s*\S*)?\"\s(\d{3})\s(-|\d+)
parsedDF: org.apache.spark.sql.DataFrame = [host: string, date: string ... 5 more fields]


**Limpio los valores nulos para el campo bytes, ya que cuando el status era 404 los bytes recibidos era nulos.**

In [6]:
val cleanDF = parsedDF
    .withColumn("bytes", when($"bytes".isNull, 0).otherwise($"bytes"))
cleanDF.cache()

cleanDF: org.apache.spark.sql.DataFrame = [host: string, date: string ... 5 more fields]
res3: cleanDF.type = [host: string, date: string ... 5 more fields]


Comprobación para ver que se había realizado bien la organización de los datos, y visto que
había filas sin protocolo, comprobar que el resto de datos estaba correcto.

In [7]:
val test = cleanDF
    .groupBy($"protocol")
    .count()
    .orderBy(desc("protocol"))
    .show(false)
cleanDF.where($"protocol" === "").show(10, false)

+-------------------------------------------------+-------+
|protocol                                         |count  |
+-------------------------------------------------+-------+
|HTTP/V1.0                                        |279    |
|HTTP/1.0From:  <berend@blazemonger.pc.cc.cmu.edu>|1235   |
|HTTP/1.0                                         |128    |
|HTTP/1.0                                         |3454985|
|HTTP/*                                           |13     |
|                                                 |4973   |
+-------------------------------------------------+-------+

+-----------------------+--------------------+------+---------------------------------------------------+--------+------+-----+
|host                   |date                |method|resource                                           |protocol|status|bytes|
+-----------------------+--------------------+------+---------------------------------------------------+--------+------+-----+
|pipe1.nyc.pipe

test: Unit = ()


In [8]:
/*https://medium.com/analytics-vidhya/spark-web-server-logs-analysis-with-scala-74e0ece40a4e

val month_map = Map("Jan" -> 1,"Feb" -> 2,"Mar" -> 3,"Apr" -> 4,"May" -> 5,"Jun" -> 6,"Jul" -> 7,
                    "Aug" -> 8,"Sep" -> 9,"Oct" -> 10,"Nov" -> 11,"Dec" -> 12)

def parse_clf_time(s: String): String ={
    "%3$s-%2$s-%1$s %4$s:%5$s:%6$s".format(s.substring(0,2), month_map(s.substring(3,6)),s.substring(7,11),
                                          s.substring(12, 14), s.substring(15,17), s.substring(18))
}

val toTimestamp = udf[String, String](parse_clf_time(_))
val logsDF = cleanDF
    .withColumn("date",to_timestamp(toTimestamp($"date")))
*/

**Cambio los valores de los meses del date a forma númerica por si necesitamos en un futuro cambiarlo a un formato de date o timestamp**

In [9]:
val logsDF = cleanDF
   .withColumn("date",to_timestamp($"date","dd/MMM/yyyy:HH:mm:ss"))

logsDF: org.apache.spark.sql.DataFrame = [host: string, date: timestamp ... 5 more fields]


**Ya tendríamos nuestro dataset limpio y listo, para realizar consultas sobre él, en este caso lo guardaremos en parquet para realizar las consultas**

In [10]:
logsDF.write.format("parquet").mode("overwrite").save("/Users/mario.serrano/Desktop/NASA/datasets/parquet/")

In [11]:
val fileParquet = "/Users/mario.serrano/Desktop/NASA/datasets/parquet/*"
val parquetDF = spark.read.format("parquet").load(fileParquet)

fileParquet: String = /Users/mario.serrano/Desktop/NASA/datasets/parquet/*
parquetDF: org.apache.spark.sql.DataFrame = [host: string, date: timestamp ... 5 more fields]


In [12]:
parquetDF.cache()
parquetDF.printSchema
parquetDF.show(10,false)

root
 |-- host: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- resource: string (nullable = true)
 |-- protocol: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- bytes: integer (nullable = true)

+--------------------+-------------------+------+-----------------------------------------------+--------+------+-----+
|host                |date               |method|resource                                       |protocol|status|bytes|
+--------------------+-------------------+------+-----------------------------------------------+--------+------+-----+
|199.72.81.55        |1995-07-01 00:00:01|GET   |/history/apollo/                               |HTTP/1.0|200   |6245 |
|unicomp6.unicomp.net|1995-07-01 00:00:06|GET   |/shuttle/countdown/                            |HTTP/1.0|200   |3985 |
|199.120.110.21      |1995-07-01 00:00:09|GET   |/shuttle/missions/sts-73/mission-sts-73.html   |HTTP/1.0|200   |4085 |
|bur

**- Dado que podemos encontrar distintos protocolos web, queremos ver cuál es el más utilizado. Para esto agrupa los protocolos y ordenalos de mayor a menor.**

In [13]:
parquetDF
    .groupBy($"protocol")
    .count()
    .orderBy(desc("count"))
    .show(false)

+-------------------------------------------------+-------+
|protocol                                         |count  |
+-------------------------------------------------+-------+
|HTTP/1.0                                         |3454985|
|                                                 |4973   |
|HTTP/1.0From:  <berend@blazemonger.pc.cc.cmu.edu>|1235   |
|HTTP/V1.0                                        |279    |
|HTTP/1.0                                         |128    |
|HTTP/*                                           |13     |
+-------------------------------------------------+-------+



Encontramos que el único protocolo utilizado es el HTTP/1.0, podríamos transformar todos los protocolos a la misma nomenclatura, también nos encontramos que hay varias entradas sin protocolo.

In [14]:
val protocolDF = parquetDF
    .withColumn("protocol",when($"protocol".contains("HTTP/"), "HTTP/1.0")
                            .when($"protocol" === "", "NA")
                            .otherwise($"protocol"))


protocolDF: org.apache.spark.sql.DataFrame = [host: string, date: timestamp ... 5 more fields]


In [15]:
protocolDF
    .groupBy($"protocol")
    .count()
    .orderBy(desc("count"))
    .show(false)

+--------+-------+
|protocol|count  |
+--------+-------+
|HTTP/1.0|3456640|
|NA      |4973   |
+--------+-------+



**- Realiza lo mismo para ver los códigos de estado y los métodos de petición que más se han realizado.**

In [16]:
parquetDF
    .groupBy($"status")
    .count()
    .orderBy(desc("count"))
    .show(false)

+------+-------+
|status|count  |
+------+-------+
|200   |3100500|
|304   |266773 |
|302   |73070  |
|404   |20866  |
|403   |225    |
|500   |65     |
|null  |60     |
|501   |41     |
|400   |13     |
+------+-------+



In [17]:
parquetDF
    .groupBy($"method")
    .count()
    .orderBy(desc("count"))
    .show(false)

+------+-------+
|method|count  |
+------+-------+
|GET   |3453403|
|HEAD  |7915   |
|POST  |222    |
|      |73     |
+------+-------+



Visto que tenemos 60 entradas con el status en null, comprobamos esos registros.

In [18]:
parquetDF.where($"status".isNull).show(60)

+----+----+------+--------+--------+------+-----+
|host|date|method|resource|protocol|status|bytes|
+----+----+------+--------+--------+------+-----+
|    |null|      |        |        |  null|    0|
|    |null|      |        |        |  null|    0|
|    |null|      |        |        |  null|    0|
|    |null|      |        |        |  null|    0|
|    |null|      |        |        |  null|    0|
|    |null|      |        |        |  null|    0|
|    |null|      |        |        |  null|    0|
|    |null|      |        |        |  null|    0|
|    |null|      |        |        |  null|    0|
|    |null|      |        |        |  null|    0|
|    |null|      |        |        |  null|    0|
|    |null|      |        |        |  null|    0|
|    |null|      |        |        |  null|    0|
|    |null|      |        |        |  null|    0|
|    |null|      |        |        |  null|    0|
|    |null|      |        |        |  null|    0|
|    |null|      |        |        |  null|    0|


Vemos que son registros vacíos, por lo que vamos a quitarlos de nuestro DataFrame.

In [19]:
val finalDF = protocolDF.na.drop(Seq("status"))
spark.catalog.clearCache()
finalDF.cache()

finalDF: org.apache.spark.sql.DataFrame = [host: string, date: timestamp ... 5 more fields]
res13: finalDF.type = [host: string, date: timestamp ... 5 more fields]


In [20]:
finalDF
    .groupBy($"status")
    .count()
    .orderBy(desc("count"))
    .show(false)

+------+-------+
|status|count  |
+------+-------+
|200   |3100500|
|304   |266773 |
|302   |73070  |
|404   |20866  |
|403   |225    |
|500   |65     |
|501   |41     |
|400   |13     |
+------+-------+



**- También queremos ver cuál es el recurso que tuvo la mayor transferencia de bytes de la página web.**

In [26]:
finalDF
    .orderBy(desc("bytes"))
    .limit(1).show(false)

+-----+-------------------+------+---------------------------------------+--------+------+-------+
|host |date               |method|resource                               |protocol|status|bytes  |
+-----+-------------------+------+---------------------------------------+--------+------+-------+
|derec|1995-07-07 14:03:32|GET   |/shuttle/countdown/video/livevideo.jpeg|HTTP/1.0|200   |6823936|
+-----+-------------------+------+---------------------------------------+--------+------+-------+



**- Además queremos saber que recurso de nuestra web es el que más tráfico recibe. Es decir, el recurso con más entradas en nuestro log.**

In [22]:
finalDF.groupBy("resource")
    .count()
    .orderBy(desc("count"))
    .show(10, false)

+---------------------------------------+------+
|resource                               |count |
+---------------------------------------+------+
|/images/NASA-logosmall.gif             |208723|
|/images/KSC-logosmall.gif              |164976|
|/images/MOSAIC-logosmall.gif           |127916|
|/images/USA-logosmall.gif              |127082|
|/images/WORLD-logosmall.gif            |125933|
|/images/ksclogo-medium.gif             |121580|
|/ksc.html                              |83912 |
|/images/launch-logo.gif                |76009 |
|/history/apollo/images/apollo-logo1.gif|68898 |
|/shuttle/countdown/                    |64739 |
+---------------------------------------+------+
only showing top 10 rows



**- También queremos saber que días fueron los que la web recibió más tráfico.**

In [23]:
finalDF
    .withColumn("day",dayofmonth($"date"))
    .withColumn("month",month($"date"))
    .groupBy("day","month")
    .count()
    .orderBy(desc("count"))
    .show(10)

+---+-----+------+
|day|month| count|
+---+-----+------+
| 13|    7|134201|
|  6|    7|100959|
|  5|    7| 94573|
| 12|    7| 92528|
| 31|    8| 90123|
|  3|    7| 89584|
|  7|    7| 87233|
| 14|    7| 84103|
| 30|    8| 80640|
| 11|    7| 80407|
+---+-----+------+
only showing top 10 rows



**- Que hosts son los más frecuentes.**

In [24]:
finalDF
    .groupBy($"host")
    .count()
    .orderBy(desc("count"))
    .show()

+--------------------+-----+
|                host|count|
+--------------------+-----+
|piweba3y.prodigy.com|21988|
|piweba4y.prodigy.com|16437|
|piweba1y.prodigy.com|12825|
|  edams.ksc.nasa.gov|11964|
|        163.206.89.4| 9697|
|         news.ti.com| 8161|
|www-d1.proxy.aol.com| 8047|
|  alyssa.prodigy.com| 8037|
| siltb10.orl.mmc.com| 7573|
|www-a2.proxy.aol.com| 7516|
|www-b2.proxy.aol.com| 7266|
|piweba2y.prodigy.com| 7246|
|www-b3.proxy.aol.com| 7218|
|www-d4.proxy.aol.com| 7211|
|www-b5.proxy.aol.com| 7080|
|www-d2.proxy.aol.com| 6984|
|www-b4.proxy.aol.com| 6972|
|www-d3.proxy.aol.com| 6895|
|    webgate1.mot.com| 6749|
|  e659229.boeing.com| 6720|
+--------------------+-----+
only showing top 20 rows

